In [ ]:
import pandas as pd
import json
import requests
from pymongo import MongoClient
from bson.objectid import ObjectId
import os
from pprint import pprint
import datetime as dt

In [ ]:
with open('D:\\lianz\\Desktop\\Python\\personal_projects\\finance_dashboard\\mongodb_api.txt','r') as f:
    cluster = f.readlines()[0]
    
client = MongoClient(cluster)

# print(client.list_database_names())

db = client.FinanceApp
balance_sheet_collection = db.balance_sheet
income_collection = db.income_statement
cash_collection = db.cash_flow_statement
company_profile = db.company_profile
historical = db.historical
stock_split = db.stock_split

In [ ]:
arr = os.listdir(r"D:\\lianz\Desktop\\Python\\personal_projects\\personal_finance\\balance-sheet-statement")
len(arr)

In [248]:
with open('D:\\lianz\\Desktop\\Python\\personal_projects\\finance_dashboard\\fmp_api.txt','r') as f:
    fmp_api = f.readlines()[0]

with open('D:\\lianz\\Desktop\\Python\\personal_projects\\finance_dashboard\\alpha_vantage_api.txt','r') as f:
    alpha_vantage_api = f.readlines()[0]

In [ ]:
def select_profile(ticker, statement):
    r = requests.get(
        f"https://financialmodelingprep.com/api/v3/{statement}/{ticker}?apikey={fmp_api}")
    r = r.json()
    return r

In [ ]:
a_profile = select_profile('A','profile')

In [ ]:
a_profile[0]['symbol']

In [ ]:
company_profile.insert_one(a_profile[0])

In [ ]:
from pymongo import ASCENDING, DESCENDING

In [ ]:
a_data = [i for i in balance_sheet_collection.find({'symbol':'AAL'}).sort('date', DESCENDING)]

In [ ]:
a_data

In [ ]:
company_profile.delete_one({})

##### Define function to access elements of an entry in database
Use function to access individual elements of a document in a collection

In [ ]:
def access_entry(collection_name, entry_name, entry_value, return_value):
    data = collection_name.find({entry_name:entry_value})

    data = [i[return_value] for i in data]

    return data

##### Get list of tickers in database

In [ ]:
balance_list = list(set([i['symbol'] for i in balance_sheet_collection.find()]))
income_list = list(set([i['symbol'] for i in income_collection.find()]))
cash_list = list(set([i['symbol'] for i in cash_collection.find()]))
company_list = list(set([i['symbol'] for i in company_profile.find()]))
list_tickers = [balance_list, income_list, cash_list, company_list]
lengths = [len(i) for i in list_tickers]
missing_tickers = [i for i in list_tickers if len(i) == max(lengths)]

In [ ]:
profiles = [select_profile(i,'profile') for i in balance_list]

In [ ]:
downloaded = [i[0]['symbol'] for i in profiles if len(i)>0]

In [ ]:
def select_quote(ticker, statement):
    r = requests.get(
        f"https://financialmodelingprep.com/api/v3/{statement}/{ticker}?limit=120&apikey=eb29218df82acef0486b5c014ccec868")
    r = r.json()
    return r

##### Insert to database from API

In [ ]:
def insert_to_mongoDB(ticker):

    balance = select_quote(ticker, 'balance-sheet-statement')
    income = select_quote(ticker, 'income-statement')
    cash_flow = select_quote(ticker, 'cash-flow-statement')

    if len(balance) <= 1 or len(income) <= 1 or len(cash_flow) <= 1:
        pass
    else:
        for i in balance:
            i['index_id'] = f"{i['symbol']}_{i['date']}"
        for i in income:
            i['index_id'] = f"{i['symbol']}_{i['date']}"    
        for i in cash_flow:
            i['index_id'] = f"{i['symbol']}_{i['date']}"
        
        bal_ids = [i['index_id'] for i in balance if i['index_id'] not in access_entry(balance_sheet_collection,'symbol',i['symbol'],'index_id')]
        income_ids = [i['index_id'] for i in income if i['index_id'] not in access_entry(income_collection,'symbol',i['symbol'],'index_id')]
        cash_ids = [i['index_id'] for i in cash_flow if i['index_id'] not in access_entry(cash_collection,'symbol',i['symbol'],'index_id')]

        try:
            balance_sheet_collection.insert_many([i for i in balance if i['index_id'] in bal_ids])
            income_collection.insert_many([i for i in income if i['index_id'] in income_ids])
            cash_collection.insert_many([i for i in cash_flow if i['index_id'] in cash_ids])
        except:
            pass

##### Insert to database from API per collection

In [ ]:
def insert_to_mongoDB(collection, ticker, statement):

    statement = select_quote(ticker, statement)

    if len(statement) <= 1:
        pass
    else:
        for i in statement:
            i['index_id'] = f"{i['symbol']}_{i['date']}"
        
        ids = [i['index_id'] for i in statement if i['index_id'] not in access_entry(collection,'symbol',i['symbol'],'index_id')]

        try:
            collection.insert_many([i for i in statement if i['index_id'] in ids])
        except:
            print('statement already exists.')

In [ ]:
insert_to_mongoDB(balance_sheet_collection, 'AAL', 'balance-sheet-statement')
insert_to_mongoDB(cash_collection, 'AAL', 'cash-flow-statement')
insert_to_mongoDB(income_collection, 'AAL', 'income-statement')

In [ ]:
# balance_sheet_collection.create_index('index_id', unique=True)
# cash_collection.create_index('index_id', unique=True)
# income_collection.create_index('index_id', unique=True)
# company_profile.create_index('index_id', unique=True)
# historical.create_index('index_id', unique=True)
stock_split.create_index('index_id', unique=True)

##### Insert to database from local computer

In [ ]:
def insert_to_mongoDB(ticker):

    balance = f'D:\\lianz\\Desktop\\Python\\personal_projects\\personal_finance\\balance-sheet-statement\\{ticker}'
    income = f'D:\\lianz\\Desktop\\Python\\personal_projects\\personal_finance\\income-statement\\{ticker}'
    cash_flow = f'D:\\lianz\\Desktop\\Python\\personal_projects\\personal_finance\\cash-flow-statement\\{ticker}'

    with open(balance, 'r') as f:
        data_bal = json.load(f)
    with open(income, 'r') as f:
        data_income = json.load(f)
    with open(cash_flow, 'r') as f:
        data_cash = json.load(f)


    if len(data_bal) <= 1 or len(data_income) <= 1 or len(data_cash) <= 1:
        pass
    else:
        for i in data_bal:
            i['index_id'] = f"{i['symbol']}_{i['date']}"
        for i in data_income:
            i['index_id'] = f"{i['symbol']}_{i['date']}"    
        for i in data_cash:
            i['index_id'] = f"{i['symbol']}_{i['date']}"
        
        bal_ids = [i['index_id'] for i in data_bal if i['index_id'] not in access_entry(balance_sheet_collection,'symbol',i['symbol'],'index_id')]
        income_ids = [i['index_id'] for i in data_bal if i['index_id'] not in access_entry(income_collection,'symbol',i['symbol'],'index_id')]
        cash_ids = [i['index_id'] for i in data_bal if i['index_id'] not in access_entry(cash_collection,'symbol',i['symbol'],'index_id')]
        
        try:
            balance_sheet_collection.insert_many([i for i in data_bal if i['index_id'] in bal_ids])
            income_collection.insert_many([i for i in data_income if i['index_id'] in income_ids])
            cash_collection.insert_many([i for i in data_cash if i['index_id'] in cash_ids])
        except:
            pass

##### Upload all currently stored statements to cloud database

In [ ]:
for i in arr:
    insert_to_mongoDB(i)

In [ ]:
# The web framework gets post_id from the URL and passes it as a string
def get(post_id):
    # Convert from string to ObjectId:
    document = client.db.collection.find_one({'_id': ObjectId(post_id)})

##### Delete database entries (only run when needed)

In [ ]:
balance_sheet_collection.delete_many({'symbol':'AF'})
cash_collection.delete_many({'symbol':'AF'})
income_collection.delete_many({'symbol':'AF'})

In [ ]:
balance_sheet_collection.delete_many({'symbol':'AAL','calendarYear':'2021'})
cash_collection.delete_many({'symbol':'AAL','calendarYear':'2021'})
income_collection.delete_many({'symbol':'AAL','calendarYear':'2021'})

In [ ]:
A_2022 = balance_sheet_collection.find({'symbol':'A'})

In [ ]:
[i for i in A_2022]

In [ ]:
balance_sheet_collection.create_index("index_id", unique=True)

In [ ]:
with open('D:\lianz\Desktop\Python\personal_projects\\finance_dashboard\\fmp_api.txt','r') as f:
    fmp_api = f.readlines()[0]

In [ ]:
r = requests.get(
    f"https://financialmodelingprep.com/api/v3/historical-price-full/CVX?apikey={fmp_api}")

r = r.json()

In [ ]:
def access_entry(_collection_name, entry_name, entry_value, return_value):
    data = _collection_name.find({entry_name: entry_value})

    data = [i[return_value] for i in data]

    return data

In [ ]:
for i in r['historical']:    
    i['index_id'] = f"{r['symbol']}_{i['date']}"
    i['symbol'] = f"{r['symbol']}"

ids = [i['index_id'] for i in r['historical'] if i['index_id']
            not in access_entry(historical, 'symbol', 'AAPL', 'index_id')]
ids

In [ ]:
try:
    historical.insert_many([i for i in r['historical'] if i['index_id'] in ids])
    print(f"{r['symbol']} historical updated!")

except:
    print(f"{r['symbol']} historical already exists")

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

df = pd.DataFrame.from_records([x for i,x in enumerate(r['historical'])], index='date')

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(
        x=df.index, y=df['adjClose'], mode="lines+markers", name=f"adjClose"),
    secondary_y=False,
)

fig.add_trace(
    go.Bar(
        x=df.index, y=df['volume'], marker=dict({'color': 'darkorange'}), texttemplate="%{value:,}", textposition="inside", name="Volume"),
    secondary_y=True,
)


In [ ]:
r['historical'][0].keys()

In [212]:
aapl_hist = pd.DataFrame.from_records([x for i,x in enumerate(historical.find({'symbol':'AAPL'}))], index='date').sort_index()

In [222]:
aapl_hist.loc[aapl_hist.index.date >= dt.date(2023,1,1)].index

DatetimeIndex(['2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06',
               '2023-01-09', '2023-01-10', '2023-01-11', '2023-01-12',
               '2023-01-13', '2023-01-17', '2023-01-18', '2023-01-19',
               '2023-01-20', '2023-01-23', '2023-01-24', '2023-01-25',
               '2023-01-26', '2023-01-27', '2023-01-30', '2023-01-31',
               '2023-02-01', '2023-02-02', '2023-02-03', '2023-02-06',
               '2023-02-07', '2023-02-08', '2023-02-09', '2023-02-10',
               '2023-02-13', '2023-02-14', '2023-02-15', '2023-02-16',
               '2023-02-17', '2023-02-21', '2023-02-22', '2023-02-23',
               '2023-02-24', '2023-02-27', '2023-02-28', '2023-03-01',
               '2023-03-02', '2023-03-03', '2023-03-07'],
              dtype='datetime64[ns]', name='date', freq=None)

In [ ]:
def download_stocksplit(ticker):
    r = requests.get(
        f"https://financialmodelingprep.com/api/v3/historical-price-full/stock_split/{ticker}?apikey={fmp_api}"
    )
    r = r.json()
    return r

In [ ]:
aapl_stocksplit = download_stocksplit('AAPL')

In [ ]:
for i in aapl_stocksplit['historical']:
    i['index_id'] = f"{aapl_stocksplit['symbol']}_{i['date']}"
    i['date'] = dt.datetime.strptime(i['date'], '%Y-%m-%d')

In [ ]:
pd.DataFrame.from_records(aapl_stocksplit['historical'], index='date')

In [ ]:
def stock_price_api(ticker):
    url = "https://alpha-vantage.p.rapidapi.com/query"
    headers = {"X-RapidAPI-Key": f"{alpha_vantage_api}",
               "X-RapidAPI-Host": "alpha-vantage.p.rapidapi.com"}
    querystring = {"function":"TIME_SERIES_DAILY","symbol":f"{ticker}","outputsize":"full","datatype":"json"}
    response = requests.request("GET", url=url, headers=headers, params=querystring)
    return response.json()

In [ ]:
historical.delete_many({'symbol':'A'})

In [ ]:
file = stock_price_api('A')


In [ ]:
for i,x in file['Time Series (Daily)'].items():    
    x['index_id'] = f"{'A'}_{i}"
    x['symbol'] = f"{'A'}"
    x['date'] = i
historical.insert_many([x for i,x in file['Time Series (Daily)'].items()])

In [ ]:
# Update date from string to datetime format
historical.update_many({'date':{'$type':"string"}},[
    {
        '$set': {
            'date': {
                '$dateFromString': {
                    'dateString': '$date', 
                    'format': '%Y-%m-%d'
                }
            }
        }
    }
])

In [ ]:
list(set([i['index_id'] for i in historical.find()]))

In [254]:
# get CPI data
## interval=monthly and semiannual are accepted.
def get_cpi(interval):
    url = f'https://www.alphavantage.co/query?function=CPI&interval={interval}&apikey={alpha_vantage_api}'
    r = requests.get(url)
    data = r.json()

    yield data

for i in get_cpi('semiannual'):
    pprint(i)

{'data': [{'date': '2022-07-01', 'value': '296.963'},
          {'date': '2022-01-01', 'value': '288.347'},
          {'date': '2021-07-01', 'value': '275.703'},
          {'date': '2021-01-01', 'value': '266.236'},
          {'date': '2020-07-01', 'value': '260.065'},
          {'date': '2020-01-01', 'value': '257.557'},
          {'date': '2019-07-01', 'value': '256.903'},
          {'date': '2019-01-01', 'value': '254.412'},
          {'date': '2018-07-01', 'value': '252.125'},
          {'date': '2018-01-01', 'value': '250.089'},
          {'date': '2017-07-01', 'value': '246.163'},
          {'date': '2017-01-01', 'value': '244.076'},
          {'date': '2016-07-01', 'value': '241.237'},
          {'date': '2016-01-01', 'value': '238.778'},
          {'date': '2015-07-01', 'value': '237.769'},
          {'date': '2015-01-01', 'value': '236.265'},
          {'date': '2014-07-01', 'value': '237.088'},
          {'date': '2014-01-01', 'value': '236.384'},
          {'date': '2013-07-

In [253]:
# get real GDP data
## interval=quarterly and annual are accepted.
def get_GDP(interval):
    url = f'https://www.alphavantage.co/query?function=REAL_GDP&interval={interval}&apikey={alpha_vantage_api}'
    r = requests.get(url)
    data = r.json()

    yield data

for i in get_GDP('annual'):
    pprint(i)

{'data': [{'date': '2022-01-01', 'value': '20015.379'},
          {'date': '2021-01-01', 'value': '19609.812'},
          {'date': '2020-01-01', 'value': '18509.143'},
          {'date': '2019-01-01', 'value': '19036.052'},
          {'date': '2018-01-01', 'value': '18609.078'},
          {'date': '2017-01-01', 'value': '18076.651'},
          {'date': '2016-01-01', 'value': '17680.274'},
          {'date': '2015-01-01', 'value': '17390.295'},
          {'date': '2014-01-01', 'value': '16932.051'},
          {'date': '2013-01-01', 'value': '16553.348'},
          {'date': '2012-01-01', 'value': '16253.97'},
          {'date': '2011-01-01', 'value': '15891.534'},
          {'date': '2010-01-01', 'value': '15648.991'},
          {'date': '2009-01-01', 'value': '15236.262'},
          {'date': '2008-01-01', 'value': '15642.962'},
          {'date': '2007-01-01', 'value': '15623.871'},
          {'date': '2006-01-01', 'value': '15315.943'},
          {'date': '2005-01-01', 'value': '14901.

In [255]:
# get treasury yield
## interval=daily, weekly, and monthly are accepted
## maturity=3month, 2year, 5year, 7year, 10year, and 30year are accepted
def get_treasury(interval, maturity):
    url = f'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval={interval}&maturity={maturity}&apikey={alpha_vantage_api}'
    r = requests.get(url)
    data = r.json()

    yield data

for i in get_treasury('monthly','2year'):
    pprint(i)

{'data': [{'date': '2023-02-01', 'value': '4.53'},
          {'date': '2023-01-01', 'value': '4.21'},
          {'date': '2022-12-01', 'value': '4.29'},
          {'date': '2022-11-01', 'value': '4.50'},
          {'date': '2022-10-01', 'value': '4.38'},
          {'date': '2022-09-01', 'value': '3.86'},
          {'date': '2022-08-01', 'value': '3.25'},
          {'date': '2022-07-01', 'value': '3.04'},
          {'date': '2022-06-01', 'value': '3.00'},
          {'date': '2022-05-01', 'value': '2.62'},
          {'date': '2022-04-01', 'value': '2.54'},
          {'date': '2022-03-01', 'value': '1.91'},
          {'date': '2022-02-01', 'value': '1.44'},
          {'date': '2022-01-01', 'value': '0.98'},
          {'date': '2021-12-01', 'value': '0.68'},
          {'date': '2021-11-01', 'value': '0.51'},
          {'date': '2021-10-01', 'value': '0.39'},
          {'date': '2021-09-01', 'value': '0.24'},
          {'date': '2021-08-01', 'value': '0.22'},
          {'date': '2021-07-01'

In [252]:
# get federal funds rate
## interval=daily, weekly, and monthly are accepted.
def get_fed_rate(interval):
    url = f'https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval={interval}&apikey={alpha_vantage_api}'
    r = requests.get(url)
    data = r.json()

    yield data

for i in get_fed_rate('monthly'):
    pprint(i)

{'data': [{'date': '2023-02-01', 'value': '4.57'},
          {'date': '2023-01-01', 'value': '4.33'},
          {'date': '2022-12-01', 'value': '4.10'},
          {'date': '2022-11-01', 'value': '3.78'},
          {'date': '2022-10-01', 'value': '3.08'},
          {'date': '2022-09-01', 'value': '2.56'},
          {'date': '2022-08-01', 'value': '2.33'},
          {'date': '2022-07-01', 'value': '1.68'},
          {'date': '2022-06-01', 'value': '1.21'},
          {'date': '2022-05-01', 'value': '0.77'},
          {'date': '2022-04-01', 'value': '0.33'},
          {'date': '2022-03-01', 'value': '0.20'},
          {'date': '2022-02-01', 'value': '0.08'},
          {'date': '2022-01-01', 'value': '0.08'},
          {'date': '2021-12-01', 'value': '0.08'},
          {'date': '2021-11-01', 'value': '0.08'},
          {'date': '2021-10-01', 'value': '0.08'},
          {'date': '2021-09-01', 'value': '0.08'},
          {'date': '2021-08-01', 'value': '0.09'},
          {'date': '2021-07-01'

In [256]:
# get retail sales data

def get_retail_sales():
    url = f'https://www.alphavantage.co/query?function=RETAIL_SALES&apikey={alpha_vantage_api}'
    r = requests.get(url)
    data = r.json()

    yield data

for i in get_retail_sales():
    pprint(i)

{'data': [{'date': '2023-01-01', 'value': '540735'},
          {'date': '2022-12-01', 'value': '657682'},
          {'date': '2022-11-01', 'value': '609862'},
          {'date': '2022-10-01', 'value': '597349'},
          {'date': '2022-09-01', 'value': '577370'},
          {'date': '2022-08-01', 'value': '613416'},
          {'date': '2022-07-01', 'value': '600746'},
          {'date': '2022-06-01', 'value': '609933'},
          {'date': '2022-05-01', 'value': '614474'},
          {'date': '2022-04-01', 'value': '594426'},
          {'date': '2022-03-01', 'value': '597077'},
          {'date': '2022-02-01', 'value': '506400'},
          {'date': '2022-01-01', 'value': '518146'},
          {'date': '2021-12-01', 'value': '632849'},
          {'date': '2021-11-01', 'value': '579687'},
          {'date': '2021-10-01', 'value': '557737'},
          {'date': '2021-09-01', 'value': '532840'},
          {'date': '2021-08-01', 'value': '553822'},
          {'date': '2021-07-01', 'value': '554

In [257]:
# get durables data
def get_durables():
    url = 'https://www.alphavantage.co/query?function=DURABLES&apikey=demo'
    r = requests.get(url)
    data = r.json()

    yield data
    
for i in get_durables():
    pprint(i)

{'data': [{'date': '2023-01-01', 'value': '250074'},
          {'date': '2022-12-01', 'value': '296687'},
          {'date': '2022-11-01', 'value': '259606'},
          {'date': '2022-10-01', 'value': '273120'},
          {'date': '2022-09-01', 'value': '289932'},
          {'date': '2022-08-01', 'value': '279323'},
          {'date': '2022-07-01', 'value': '249456'},
          {'date': '2022-06-01', 'value': '294470'},
          {'date': '2022-05-01', 'value': '263971'},
          {'date': '2022-04-01', 'value': '260366'},
          {'date': '2022-03-01', 'value': '294088'},
          {'date': '2022-02-01', 'value': '250105'},
          {'date': '2022-01-01', 'value': '242492'},
          {'date': '2021-12-01', 'value': '267477'},
          {'date': '2021-11-01', 'value': '244327'},
          {'date': '2021-10-01', 'value': '247086'},
          {'date': '2021-09-01', 'value': '260121'},
          {'date': '2021-08-01', 'value': '250971'},
          {'date': '2021-07-01', 'value': '228

In [259]:
def get_unemployment():
    url = f'https://www.alphavantage.co/query?function=UNEMPLOYMENT&apikey={alpha_vantage_api}'
    r = requests.get(url)
    data = r.json()

    yield data

for i in get_unemployment():
    pprint(i)

{'data': [{'date': '2023-01-01', 'value': '3.4'},
          {'date': '2022-12-01', 'value': '3.5'},
          {'date': '2022-11-01', 'value': '3.6'},
          {'date': '2022-10-01', 'value': '3.7'},
          {'date': '2022-09-01', 'value': '3.5'},
          {'date': '2022-08-01', 'value': '3.7'},
          {'date': '2022-07-01', 'value': '3.5'},
          {'date': '2022-06-01', 'value': '3.6'},
          {'date': '2022-05-01', 'value': '3.6'},
          {'date': '2022-04-01', 'value': '3.6'},
          {'date': '2022-03-01', 'value': '3.6'},
          {'date': '2022-02-01', 'value': '3.8'},
          {'date': '2022-01-01', 'value': '4.0'},
          {'date': '2021-12-01', 'value': '3.9'},
          {'date': '2021-11-01', 'value': '4.2'},
          {'date': '2021-10-01', 'value': '4.5'},
          {'date': '2021-09-01', 'value': '4.8'},
          {'date': '2021-08-01', 'value': '5.2'},
          {'date': '2021-07-01', 'value': '5.4'},
          {'date': '2021-06-01', 'value': '5.9'},


In [261]:
def get_nonfarm_payroll():
    url = f'https://www.alphavantage.co/query?function=NONFARM_PAYROLL&apikey={alpha_vantage_api}'
    r = requests.get(url)
    data = r.json()

    yield data

for i in get_nonfarm_payroll():
    pprint(i)

{'data': [{'date': '2023-01-01', 'value': '152844'},
          {'date': '2022-12-01', 'value': '155349'},
          {'date': '2022-11-01', 'value': '155642'},
          {'date': '2022-10-01', 'value': '155041'},
          {'date': '2022-09-01', 'value': '153809'},
          {'date': '2022-08-01', 'value': '153285'},
          {'date': '2022-07-01', 'value': '152875'},
          {'date': '2022-06-01', 'value': '153217'},
          {'date': '2022-05-01', 'value': '152291'},
          {'date': '2022-04-01', 'value': '151449'},
          {'date': '2022-03-01', 'value': '150411'},
          {'date': '2022-02-01', 'value': '149606'},
          {'date': '2022-01-01', 'value': '147932'},
          {'date': '2021-12-01', 'value': '150740'},
          {'date': '2021-11-01', 'value': '150543'},
          {'date': '2021-10-01', 'value': '149605'},
          {'date': '2021-09-01', 'value': '147917'},
          {'date': '2021-08-01', 'value': '147159'},
          {'date': '2021-07-01', 'value': '146